<a href="https://colab.research.google.com/github/nitinn01/Kaggle-Fashion_Mnist-Dataset/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [13]:
!pip install keras_tuner

In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.5.0


In [17]:
fashion_mnist=keras.datasets.fashion_mnist

In [18]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [19]:
train_images=train_images/255.0
test_images=test_images/255.0

In [20]:
train_images[0].shape

(28, 28)

In [21]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [22]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [23]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [24]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [25]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 22s]
val_accuracy: 0.9141666889190674

Best val_accuracy So Far: 0.9156666398048401
Total elapsed time: 00h 03m 09s
INFO:tensorflow:Oracle triggered exit


In [26]:
model=tuner_search.get_best_models(num_models=1)[0]

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        55360     
_________________________________________________________________
flatten (Flatten)            (None, 36864)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2949200   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 3,006,330
Trainable params: 3,006,330
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1180 - accuracy: 0.9564 - val_loss: 0.2615 - val_accuracy: 0.9150
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0807 - accuracy: 0.9697 - val_loss: 0.2964 - val_accuracy: 0.9150
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0572 - accuracy: 0.9787 - val_loss: 0.3480 - val_accuracy: 0.9117
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0404 - accuracy: 0.9850 - val_loss: 0.3733 - val_accuracy: 0.9173
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0324 - accuracy: 0.9889 - val_loss: 0.4634 - val_accuracy: 0.9163
Epoch 9/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0235 - accuracy: 0.9917 - val_loss: 0.5508 - val_accuracy: 0.9093
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0202 - accuracy: 0.9930 - val_loss: 0.4621 - val_accuracy